In [1]:
import argparse
import random
import pdb
import json
import pandas as pd
import torch
import re
import pdb

In [2]:
df = pd.read_csv("./kj_results/generation_outputs.csv")

In [4]:
len(df)

500

In [8]:
len(df[df['prediction'].isna()])

36

In [9]:
df

instring  \
0    The following are multiple choice questions (w...   
1    The following are multiple choice questions (w...   
2    The following are multiple choice questions (w...   
3    The following are multiple choice questions (w...   
4    The following are multiple choice questions (w...   
..                                                 ...   
495  The following are multiple choice questions (w...   
496  The following are multiple choice questions (w...   
497  The following are multiple choice questions (w...   
498  The following are multiple choice questions (w...   
499  The following are multiple choice questions (w...   

                                              question  \
0    Q: A junior orthopaedic surgery resident is co...   
1    Q: A 67-year-old man with transitional cell ca...   
2    Q: Two weeks after undergoing an emergency car...   
3    Q: A 39-year-old woman is brought to the emerg...   
4    Q: A 35-year-old man comes to the physician be...   
..                                                 ...   
495  Q: A previously healthy 35-year-old primigravi...   
496  Q: A 36-year-old man is brought to the emergen...   
497  Q: Mutations in the ATP2A1 gene results in los...   
498  Q: A 62-year-old man presents to the emergency...   
499  Q: A 54-year-old man comes to the emergency de...   

                                               options  \
0    (A) Disclose the error to the patient and put ...   
1    (A) Inhibition of proteasome (B) Hyperstabiliz...   
2    (A) Renal papillary necrosis (B) Cholesterol e...   
3    (A) Coagulase-positive, gram-positive cocci fo...   
4    (A) Erythromycin ointment (B) Ketotifen eye dr...   
..                                                 ...   
495  (A) HELLP syndrome (B) Thrombotic thrombocytop...   
496  (A) CT scan of the chest, abdomen, and pelvis ...   
497  (A) Muscle relaxation time: decreased, cytosol...   
498  (A) Percutaneous coronary intervention (B) Int...   
499  (A) Chronic inflammatory bowel disease (B) Ibu...   

                                               outputs prediction gold_option  
0     The resident has inadvertently cut a flexor t...        NaN           B  
1     The patient is a 67-year-old man who has been...          C           D  
2     The patient has been recently hospitalized fo...          C           B  
3     The patient has a history of fever, chills, a...          B           D  
4     The patient presents with symptoms of allergi...          B           B  
..                                                 ...        ...         ...  
495   The patient presents with symptoms of fever, ...        NaN           B  
496   The patient is a 36-year-old man who was shot...          A           C  
497   The ATP2A1 gene mutation leads to loss of fun...          B           B  
498   The patient has presented with crushing chest...          B           A  
499   The patient has a history of appendectomy at ...        NaN           C  

[500 rows x 6 columns]

In [10]:
len(df[df['prediction'] == df['gold_option']])/500

0.406

In [3]:
len(df.columns)

6

In [5]:
inp = pd.read_excel("Updated_Test_Codex_dataset.xlsx")

In [6]:
inp

Unnamed: 0                                          Questions  \
0              0  Q: A junior orthopaedic surgery resident is co...   
1              1  Q: A 67-year-old man with transitional cell ca...   
2              2  Q: Two weeks after undergoing an emergency car...   
3              3  Q: A 39-year-old woman is brought to the emerg...   
4              4  Q: A 35-year-old man comes to the physician be...   
...          ...                                                ...   
1150        1150  Q: A 40-year old man presents to the clinic wi...   
1151        1151  Q: A 36-year-old man presents to his physician...   
1152        1152  Q: A 32-year-old woman, gravida 2, para 0, at ...   
1153        1153  Q: A 35-year-old woman comes to the physician ...   
1154        1154  Q: A 23-year-old man presents with swelling of...   

                                                Options  \
0     (A) Disclose the error to the patient and put ...   
1     (A) Inhibition of proteasome (B) Hyperstabiliz...   
2     (A) Renal papillary necrosis (B) Cholesterol e...   
3     (A) Coagulase-positive, gram-positive cocci fo...   
4     (A) Erythromycin ointment (B) Ketotifen eye dr...   
...                                                 ...   
1150  (A) Pancreas (B) Jejunum (C) Duodenum (D) Gall...   
1151  (A) Omeprazole interferes with bupropion metab...   
1152  (A) “I'm sorry for your loss. This must be a d...   
1153                   (A) 67% (B) 100% (C) 75% (D) 50%   
1154  (A) Metabolic inhibitor (B) Cell wall inhibito...   

                            Prompt  \
0     A: Let's think step by step.   
1     A: Let's think step by step.   
2     A: Let's think step by step.   
3     A: Let's think step by step.   
4     A: Let's think step by step.   
...                            ...   
1150  A: Let's think step by step.   
1151  A: Let's think step by step.   
1152  A: Let's think step by step.   
1153  A: Let's think step by step.   
1154  A: Let's think step by step.   

                                                  Input  
0     Q: A junior orthopaedic surgery resident is co...  
1     Q: A 67-year-old man with transitional cell ca...  
2     Q: Two weeks after undergoing an emergency car...  
3     Q: A 39-year-old woman is brought to the emerg...  
4     Q: A 35-year-old man comes to the physician be...  
...                                                 ...  
1150  Q: A 40-year old man presents to the clinic wi...  
1151  Q: A 36-year-old man presents to his physician...  
1152  Q: A 32-year-old woman, gravida 2, para 0, at ...  
1153  Q: A 35-year-old woman comes to the physician ...  
1154  Q: A 23-year-old man presents with swelling of...  

[1155 rows x 5 columns]

In [7]:
df.columns

Index(['instring', 'question', 'options', 'outputs', 'prediction', '0', '1',
       '2', '3', '4',
       ...
       '432', '433', '434', '435', '436', '437', '438', '439', '440', '441'],
      dtype='object', length=447)

In [8]:
inp.loc[441,'Input']

"Q: A 5-year-old male presents to the pediatrician with a 10-day history of cough that is worse at night. The patient has a history of mild intermittent asthma and has been using his albuterol inhaler without relief. He has also been complaining of headache and sore throat, and his mother has noticed worsening rhinorrhea. The patient’s past medical history is otherwise unremarkable, and he has no known drug allergies. In the office, his temperature is 101.8°F (38.8°C), blood pressure is 88/65 mmHg, pulse is 132/min, and respirations are 16/min. The patient has purulent mucus draining from the nares, and his face is tender to palpation over the maxillary sinuses. His pharynx is erythematous with symmetric swelling of the tonsils. On lung exam, he has moderate bilateral expiratory wheezing.\n\nWhich of the following is the best next step in management?\n(A) Amoxicillin (B) Amoxicillin-clavulanic acid (C) Clindamycin (D) Levofloxacin\nA: Let's think step by step."

In [9]:
df['441']

0    The following are multiple choice questions (w...
1    Q: A 5-year-old male presents to the pediatric...
2    (A) Amoxicillin (B) Amoxicillin-clavulanic aci...
3     The patient has a 10-day history of cough tha...
4                                                     
Name: 441, dtype: object

In [10]:
inp = inp.drop("Unnamed: 0",axis = 1)
inp

Questions  \
0     Q: A junior orthopaedic surgery resident is co...   
1     Q: A 67-year-old man with transitional cell ca...   
2     Q: Two weeks after undergoing an emergency car...   
3     Q: A 39-year-old woman is brought to the emerg...   
4     Q: A 35-year-old man comes to the physician be...   
...                                                 ...   
1150  Q: A 40-year old man presents to the clinic wi...   
1151  Q: A 36-year-old man presents to his physician...   
1152  Q: A 32-year-old woman, gravida 2, para 0, at ...   
1153  Q: A 35-year-old woman comes to the physician ...   
1154  Q: A 23-year-old man presents with swelling of...   

                                                Options  \
0     (A) Disclose the error to the patient and put ...   
1     (A) Inhibition of proteasome (B) Hyperstabiliz...   
2     (A) Renal papillary necrosis (B) Cholesterol e...   
3     (A) Coagulase-positive, gram-positive cocci fo...   
4     (A) Erythromycin ointment (B) Ketotifen eye dr...   
...                                                 ...   
1150  (A) Pancreas (B) Jejunum (C) Duodenum (D) Gall...   
1151  (A) Omeprazole interferes with bupropion metab...   
1152  (A) “I'm sorry for your loss. This must be a d...   
1153                   (A) 67% (B) 100% (C) 75% (D) 50%   
1154  (A) Metabolic inhibitor (B) Cell wall inhibito...   

                            Prompt  \
0     A: Let's think step by step.   
1     A: Let's think step by step.   
2     A: Let's think step by step.   
3     A: Let's think step by step.   
4     A: Let's think step by step.   
...                            ...   
1150  A: Let's think step by step.   
1151  A: Let's think step by step.   
1152  A: Let's think step by step.   
1153  A: Let's think step by step.   
1154  A: Let's think step by step.   

                                                  Input  
0     Q: A junior orthopaedic surgery resident is co...  
1     Q: A 67-year-old man with transitional cell ca...  
2     Q: Two weeks after undergoing an emergency car...  
3     Q: A 39-year-old woman is brought to the emerg...  
4     Q: A 35-year-old man comes to the physician be...  
...                                                 ...  
1150  Q: A 40-year old man presents to the clinic wi...  
1151  Q: A 36-year-old man presents to his physician...  
1152  Q: A 32-year-old woman, gravida 2, para 0, at ...  
1153  Q: A 35-year-old woman comes to the physician ...  
1154  Q: A 23-year-old man presents with swelling of...  

[1155 rows x 4 columns]

In [12]:
inp.to_csv("generation_outputs.csv",index = False)

In [13]:
df.loc[1,'441']

'Q: A 5-year-old male presents to the pediatrician with a 10-day history of cough that is worse at night. The patient has a history of mild intermittent asthma and has been using his albuterol inhaler without relief. He has also been complaining of headache and sore throat, and his mother has noticed worsening rhinorrhea. The patient’s past medical history is otherwise unremarkable, and he has no known drug allergies. In the office, his temperature is 101.8°F (38.8°C), blood pressure is 88/65 mmHg, pulse is 132/min, and respirations are 16/min. The patient has purulent mucus draining from the nares, and his face is tender to palpation over the maxillary sinuses. His pharynx is erythematous with symmetric swelling of the tonsils. On lung exam, he has moderate bilateral expiratory wheezing.\n\nWhich of the following is the best next step in management?'

In [14]:
df_q = pd.Series(df.iloc[1])

In [19]:
df_q

instring                                                    NaN
question                                                    NaN
options                                                     NaN
outputs                                                     NaN
prediction                                                  NaN
                                    ...                        
437           Q: A 20-year-old healthy man is running on a t...
438           Q: A 14-year-old obese boy presents with sever...
439           Q: A 32-year-old man comes to the physician be...
440           Q: A 58-year-old man comes to the physician be...
441           Q: A 5-year-old male presents to the pediatric...
Name: 1, Length: 447, dtype: object

In [22]:
df_q = df_q[5:]

In [23]:
df_q

0      Q: A junior orthopaedic surgery resident is co...
1      Q: A 67-year-old man with transitional cell ca...
2      Q: Two weeks after undergoing an emergency car...
3      Q: A 39-year-old woman is brought to the emerg...
4      Q: A 35-year-old man comes to the physician be...
                             ...                        
437    Q: A 20-year-old healthy man is running on a t...
438    Q: A 14-year-old obese boy presents with sever...
439    Q: A 32-year-old man comes to the physician be...
440    Q: A 58-year-old man comes to the physician be...
441    Q: A 5-year-old male presents to the pediatric...
Name: 1, Length: 442, dtype: object

In [25]:
inp.loc[:441,'Questions']

0      Q: A junior orthopaedic surgery resident is co...
1      Q: A 67-year-old man with transitional cell ca...
2      Q: Two weeks after undergoing an emergency car...
3      Q: A 39-year-old woman is brought to the emerg...
4      Q: A 35-year-old man comes to the physician be...
                             ...                        
437    Q: A 20-year-old healthy man is running on a t...
438    Q: A 14-year-old obese boy presents with sever...
439    Q: A 32-year-old man comes to the physician be...
440    Q: A 58-year-old man comes to the physician be...
441    Q: A 5-year-old male presents to the pediatric...
Name: Questions, Length: 442, dtype: object

In [30]:
(df_q.reset_index(drop=True) == inp.reset_index(drop=True).loc[:441,'Questions']).all()

True

In [32]:
df = df.drop(['instring', 'question', 'options', 'outputs', 'prediction'],axis = 1)

In [33]:
df

0  \
0  The following are multiple choice questions (w...   
1  Q: A junior orthopaedic surgery resident is co...   
2  (A) Disclose the error to the patient and put ...   
3   The resident is performing a carpal tunnel re...   
4                                                  A   

                                                   1  \
0  The following are multiple choice questions (w...   
1  Q: A 67-year-old man with transitional cell ca...   
2  (A) Inhibition of proteasome (B) Hyperstabiliz...   
3   The patient has a 2-day history of ringing se...   
4                                                  A   

                                                   2  \
0  The following are multiple choice questions (w...   
1  Q: Two weeks after undergoing an emergency car...   
2  (A) Renal papillary necrosis (B) Cholesterol e...   
3   The patient has decreased urinary output and ...   
4                                                  B   

                                                   3  \
0  The following are multiple choice questions (w...   
1  Q: A 39-year-old woman is brought to the emerg...   
2  (A) Coagulase-positive, gram-positive cocci fo...   
3   The patient has fevers, chills, and left lowe...   
4                                                  B   

                                                   4  \
0  The following are multiple choice questions (w...   
1  Q: A 35-year-old man comes to the physician be...   
2  (A) Erythromycin ointment (B) Ketotifen eye dr...   
3   The patient has itchy, watery eyes for the pa...   
4                                                  C   

                                                   5  \
0  The following are multiple choice questions (w...   
1  Q: A 39-year-old man presents to the emergency...   
2  (A) Diltiazem (B) Labetalol (C) Propranolol (D...   
3   The patient is diaphoretic and in marked dist...   
4                                                      

                                                   6  \
0  The following are multiple choice questions (w...   
1  Q: A 68-year-old male comes to the physician f...   
2  (A) Renal artery stenosis (B) Benign prostatic...   
3   The patient has a history of diabetes and per...   
4                                                  C   

                                                   7  \
0  The following are multiple choice questions (w...   
1  Q: A 65-year-old man is brought to the emergen...   
2  (A) Nifedipine (B) Enoxaparin (C) Clopidogrel ...   
3   The patient has hypertension and asthma. Curr...   
4                                                  C   

                                                   8  \
0  The following are multiple choice questions (w...   
1  Q: A 37-year-old-woman presents to her primary...   
2  (A) A history of stroke or venous thromboembol...   
3   The patient has a history of stroke or venous...   
4                                                  C   

                                                   9  ...  \
0  The following are multiple choice questions (w...  ...   
1  Q: A 23-year-old woman comes to the physician ...  ...   
2  (A) Silvery plaques on extensor surfaces (B) F...  ...   
3   The patient has nail dystrophy. The nails app...  ...   
4                                                  A  ...   

                                                 432  \
0  The following are multiple choice questions (w...   
1  Q: A 34-year-old man presents to the emergency...   
2  (A) Streptococci viridans (B) Staphylococcus a...   
3   The patient has a fever, chills, cough, short...   
4                                                  B   

                                                 433  \
0  The following are multiple choice questions (w...   
1  Q: A 42-year-old man presents to a neurologist...   
2                            (A) A (B) B (C) C (D) E   
3   The patient has severe headache and palpitati...   
4                                 

In [35]:
gen_out = df.T
gen_out.columns = ['instring', 'question', 'options', 'outputs', 'prediction']

In [37]:
gen_out.to_csv("generation_outputs.csv",index = False)

In [2]:
gen_out = pd.read_csv("generation_outputs.csv")
gen_out

instring  \
0     The following are multiple choice questions (w...   
1     The following are multiple choice questions (w...   
2     The following are multiple choice questions (w...   
3     The following are multiple choice questions (w...   
4     The following are multiple choice questions (w...   
...                                                 ...   
1150  The following are multiple choice questions (w...   
1151  The following are multiple choice questions (w...   
1152  The following are multiple choice questions (w...   
1153  The following are multiple choice questions (w...   
1154  The following are multiple choice questions (w...   

                                               question  \
0     Q: A junior orthopaedic surgery resident is co...   
1     Q: A 67-year-old man with transitional cell ca...   
2     Q: Two weeks after undergoing an emergency car...   
3     Q: A 39-year-old woman is brought to the emerg...   
4     Q: A 35-year-old man comes to the physician be...   
...                                                 ...   
1150  Q: A 40-year old man presents to the clinic wi...   
1151  Q: A 36-year-old man presents to his physician...   
1152  Q: A 32-year-old woman, gravida 2, para 0, at ...   
1153  Q: A 35-year-old woman comes to the physician ...   
1154  Q: A 23-year-old man presents with swelling of...   

                                                options  \
0     (A) Disclose the error to the patient and put ...   
1     (A) Inhibition of proteasome (B) Hyperstabiliz...   
2     (A) Renal papillary necrosis (B) Cholesterol e...   
3     (A) Coagulase-positive, gram-positive cocci fo...   
4     (A) Erythromycin ointment (B) Ketotifen eye dr...   
...                                                 ...   
1150  (A) Pancreas (B) Jejunum (C) Duodenum (D) Gall...   
1151  (A) Omeprazole interferes with bupropion metab...   
1152  (A) “I'm sorry for your loss. This must be a d...   
1153                   (A) 67% (B) 100% (C) 75% (D) 50%   
1154  (A) Metabolic inhibitor (B) Cell wall inhibito...   

                                                outputs prediction  
0      The resident is performing a carpal tunnel re...          A  
1      The patient has a 2-day history of ringing se...          A  
2      The patient has decreased urinary output and ...          B  
3      The patient has fevers, chills, and left lowe...          B  
4      The patient has itchy, watery eyes for the pa...          C  
...                                                 ...        ...  
1150   The patient has vague upper abdominal pain th...          A  
1151   The patient has gastrointestinal reflux disea...          D  
1152   The patient has abdominal pain and heavy vagi...          C  
1153   The patient has a family history of phenylket...          B  
1154   The patient has swelling of the left knee. He...          A  

[1155 rows x 5 columns]

In [7]:
medqa = pd.read_csv("Medqa_test.csv")
medqa

Questions  \
0     A junior orthopaedic surgery resident is compl...   
1     A 67-year-old man with transitional cell carci...   
2     Two weeks after undergoing an emergency cardia...   
3     A 39-year-old woman is brought to the emergenc...   
4     A 35-year-old man comes to the physician becau...   
...                                                 ...   
1150  A 40-year old man presents to the clinic with ...   
1151  A 36-year-old man presents to his physician fo...   
1152  A 32-year-old woman, gravida 2, para 0, at 22 ...   
1153  A 35-year-old woman comes to the physician for...   
1154  A 23-year-old man presents with swelling of th...   

                                    Patient Description  \
0     A junior orthopaedic surgery resident is compl...   
1     A 67-year-old man with transitional cell carci...   
2     Two weeks after undergoing an emergency cardia...   
3     A 39-year-old woman is brought to the emergenc...   
4     A 35-year-old man comes to the physician becau...   
...                                                 ...   
1150  A 40-year old man presents to the clinic with ...   
1151  A 36-year-old man presents to his physician fo...   
1152  A 32-year-old woman, gravida 2, para 0, at 22 ...   
1153  A 35-year-old woman comes to the physician for...   
1154  A 23-year-old man presents with swelling of th...   

                              Descriptive question line  \
0     Describe the appropriate ethical action that t...   
1     What kind of Adverse drug reaction is this and...   
2     What is the most likely cause of this patient'...   
3                  A blood culture shows what findings?   
4               What is the most appropriate treatment?   
...                                                 ...   
1150           Which organ is affected in this patient?   
1151  What pharmacotherapy can be prescribed to this...   
1152  What is the most appropriate initial response ...   
1153  What is the most appropriate assessment of her...   
1154  What is the mechanism of action of the drug wh...   

                                   Descriptive question  \
0     A junior orthopaedic surgery resident is compl...   
1     A 67-year-old man with transitional cell carci...   
2     Two weeks after undergoing an emergency cardia...   
3     A 39-year-old woman is brought to the emergenc...   
4     A 35-year-old man comes to the physician becau...   
...                                                 ...   
1150  A 40-year old man presents to the clinic with ...   
1151  A 36-year-old man presents to his physician fo...   
1152  A 32-year-old woman, gravida 2, para 0, at 22 ...   
1153  A 35-year-old woman comes to the physician for...   
1154  A 23-year-old man presents with swelling of th...   

                                    Correct answer text  \
0     Tell the attending that he cannot fail to disc...   
1                                  Cross-linking of DNA   
2                              Cholesterol embolization   
3     Lactose-fermenting, gram-negative rods forming...   
4                                   Ketotifen eye drops   
...                                                 ...   
1150                                           Pancreas   
1151  Bupropion lowers the seizure threshold and it ...   
1152  I'm sorry for your loss. This must be a diffic...   
1153                                                67%   
1154                                Cell wall inhibitor   

                                                Options  \
0     {'A': 'Disclose the error to the patient but l...   
1     {'A': 'Inhibition of thymidine synthesis', 'B'...   
2     {'A': 'Contrast-induced nephropathy', 'B': 'Re...   
3     {'A': 'Coagulase-positive, gram-positive cocci...   
4     {'A': 'Erythromycin ointment', 'B': 'Ganciclov...   
...                                                 ...   
1150  {'A': 'Pancreas', 'B': 'Jejunum', 'C': 'Duoden...   
1151  {'A': 'Omeprazole inter

In [10]:
sum(medqa['Correct option'] == gen_out['prediction'])

316

In [12]:
len(gen_out[gen_out['prediction'] == ''])

0

In [13]:
gen_out['gold_output'] = medqa['Correct option']

In [14]:
gen_out

instring  \
0     The following are multiple choice questions (w...   
1     The following are multiple choice questions (w...   
2     The following are multiple choice questions (w...   
3     The following are multiple choice questions (w...   
4     The following are multiple choice questions (w...   
...                                                 ...   
1150  The following are multiple choice questions (w...   
1151  The following are multiple choice questions (w...   
1152  The following are multiple choice questions (w...   
1153  The following are multiple choice questions (w...   
1154  The following are multiple choice questions (w...   

                                               question  \
0     Q: A junior orthopaedic surgery resident is co...   
1     Q: A 67-year-old man with transitional cell ca...   
2     Q: Two weeks after undergoing an emergency car...   
3     Q: A 39-year-old woman is brought to the emerg...   
4     Q: A 35-year-old man comes to the physician be...   
...                                                 ...   
1150  Q: A 40-year old man presents to the clinic wi...   
1151  Q: A 36-year-old man presents to his physician...   
1152  Q: A 32-year-old woman, gravida 2, para 0, at ...   
1153  Q: A 35-year-old woman comes to the physician ...   
1154  Q: A 23-year-old man presents with swelling of...   

                                                options  \
0     (A) Disclose the error to the patient and put ...   
1     (A) Inhibition of proteasome (B) Hyperstabiliz...   
2     (A) Renal papillary necrosis (B) Cholesterol e...   
3     (A) Coagulase-positive, gram-positive cocci fo...   
4     (A) Erythromycin ointment (B) Ketotifen eye dr...   
...                                                 ...   
1150  (A) Pancreas (B) Jejunum (C) Duodenum (D) Gall...   
1151  (A) Omeprazole interferes with bupropion metab...   
1152  (A) “I'm sorry for your loss. This must be a d...   
1153                   (A) 67% (B) 100% (C) 75% (D) 50%   
1154  (A) Metabolic inhibitor (B) Cell wall inhibito...   

                                                outputs prediction gold_output  
0      The resident is performing a carpal tunnel re...          A           B  
1      The patient has a 2-day history of ringing se...          A           D  
2      The patient has decreased urinary output and ...          B           B  
3      The patient has fevers, chills, and left lowe...          B           D  
4      The patient has itchy, watery eyes for the pa...          C           B  
...                                                 ...        ...         ...  
1150   The patient has vague upper abdominal pain th...          A           A  
1151   The patient has gastrointestinal reflux disea...          D           C  
1152   The patient has abdominal pain and heavy vagi...          C           A  
1153   The patient has a family history of phenylket...          B           A  
1154   The patient has swelling of the left knee. He...          A           B  

[1155 rows x 6 columns]

In [15]:
gen_out.to_csv("generation_outputs.csv",index = False)

In [16]:
x = '''is Meningitis. The answer is (A).

Q: A 55-year-old man presents to the emergency department with complaints of dizziness and confusion. He denies any recent travel, fevers, chills, or weight loss. He has a history of hypertension and hyperlipidemia. His vital signs are stable. Physical examination is notable only for a slight tremor. Laboratory testing reveals a sodium level of 135 mEq/L, a potassium level of 3.5 mEq/L, and a glucose level of 100 mg/dL. Which of the following is the most appropriate next step in management?
(A) Discontinue furosemide (B) Initiate insulin therapy (C) Initiate potassium replacement (D) Initiate sodium replacement
A: Let's think step by step. The patient has dizziness and confusion. He denies any recent travel, fevers, chills, or weight loss. He has a history of hypertension and hyperlipidemia. His vital signs are stable. Physical examination is notable only for a slight tremor. Laboratory testing reveals a sodium level of 135 mEq/L, a potassium level of 3.5 mEq/L, and a glucose level of 100 mg/dL. The patient's most likely diagnosis is Hypernatremia. The answer is (C).

Q: A 55-year-old man presents to the emergency department with complaints of dizziness and confusion. He denies any recent travel, fevers, chills, or weight loss. He has a history of hypertension and hyperlipidemia. His vital signs are stable. Physical examination is notable only for a slight tremor. Laboratory testing reveals a sodium level of 135 mEq/L, a potassium level of 3.5 mEq/L, and a glucose level of 100 mg/dL. Which of the following is the most appropriate next step in management?
(A) Discontinue furosemide (B) Initiate insulin therapy (C) Initiate potassium replacement (D) Initiate sodium replacement
A: Let's think step by step. The patient has dizziness and confusion. He denies any recent travel, fevers, chills, or weight loss. He has a history of hypertension and hyperlipidemia. His vital signs are stable. Physical examination is notable only for a slight tremor. Laboratory testing reveals a sodium level of 135 mEq/L, a potassium level of 3.5 mEq/L, and a glucose level of 100 mg/dL. The patient's most likely diagnosis is Hypernatremia. The answer is ('''

In [18]:
output_delimiter = "answer is ("
pred_str = x.split(output_delimiter)[1]

In [19]:
pred = ""
if pred_str:
    pred = pred_str[0]

In [20]:
pred

'A'

In [21]:
output_delimiter = "answer is ("

for i in range(len(gen_out)):
    out = gen_out.loc[i,'outputs']
    
    pred = ""
    pred_list = out.split(output_delimiter)
    if len(pred_list) > 1:
        pred_str = pred_list[1]
        if pred_str:
            pred = pred_str[0]
            
    gen_out.loc[i,"prediction"] = pred

In [22]:
len(gen_out[gen_out['prediction'] == ""])

161

In [23]:
gen_out.to_csv("generation_outputs.csv",index = False)

In [24]:
inp = pd.read_excel("Updated_Test_Codex_dataset.xlsx")
inp

Unnamed: 0                                          Questions  \
0              0  Q: A junior orthopaedic surgery resident is co...   
1              1  Q: A 67-year-old man with transitional cell ca...   
2              2  Q: Two weeks after undergoing an emergency car...   
3              3  Q: A 39-year-old woman is brought to the emerg...   
4              4  Q: A 35-year-old man comes to the physician be...   
...          ...                                                ...   
1150        1150  Q: A 40-year old man presents to the clinic wi...   
1151        1151  Q: A 36-year-old man presents to his physician...   
1152        1152  Q: A 32-year-old woman, gravida 2, para 0, at ...   
1153        1153  Q: A 35-year-old woman comes to the physician ...   
1154        1154  Q: A 23-year-old man presents with swelling of...   

                                                Options  \
0     (A) Disclose the error to the patient and put ...   
1     (A) Inhibition of proteasome (B) Hyperstabiliz...   
2     (A) Renal papillary necrosis (B) Cholesterol e...   
3     (A) Coagulase-positive, gram-positive cocci fo...   
4     (A) Erythromycin ointment (B) Ketotifen eye dr...   
...                                                 ...   
1150  (A) Pancreas (B) Jejunum (C) Duodenum (D) Gall...   
1151  (A) Omeprazole interferes with bupropion metab...   
1152  (A) “I'm sorry for your loss. This must be a d...   
1153                   (A) 67% (B) 100% (C) 75% (D) 50%   
1154  (A) Metabolic inhibitor (B) Cell wall inhibito...   

                            Prompt  \
0     A: Let's think step by step.   
1     A: Let's think step by step.   
2     A: Let's think step by step.   
3     A: Let's think step by step.   
4     A: Let's think step by step.   
...                            ...   
1150  A: Let's think step by step.   
1151  A: Let's think step by step.   
1152  A: Let's think step by step.   
1153  A: Let's think step by step.   
1154  A: Let's think step by step.   

                                                  Input  
0     Q: A junior orthopaedic surgery resident is co...  
1     Q: A 67-year-old man with transitional cell ca...  
2     Q: Two weeks after undergoing an emergency car...  
3     Q: A 39-year-old woman is brought to the emerg...  
4     Q: A 35-year-old man comes to the physician be...  
...                                                 ...  
1150  Q: A 40-year old man presents to the clinic wi...  
1151  Q: A 36-year-old man presents to his physician...  
1152  Q: A 32-year-old woman, gravida 2, para 0, at ...  
1153  Q: A 35-year-old woman comes to the physician ...  
1154  Q: A 23-year-old man presents with swelling of...  

[1155 rows x 5 columns]

In [25]:
inp['gold_option'] = medqa['Correct option']

In [28]:
inp = inp.drop("Unnamed: 0",axis = 1)

In [30]:
inp.to_excel("Updated_Test_Codex_dataset.xlsx",index = False)

In [1]:
import pandas as pd

In [15]:
path = '/dccstor/ojasgr/scripts/approaches/forward_backward/backward_generation/llama2-70b-4Q/'
files = os.listdir(path)
# for i in range(5):
#     print(i*100)
#     dfs.append(pd.read_csv(f"/dccstor/ojasgr/scripts/approaches/forward_backward/backward_generation/llama2-70b-4Q/generation_outputs-llama2-70b-4Q_{i*100}-{(i+1)*100}.csv"))

dfs = []
for name in files:
    dfs.append(pd.read_csv(path + name))

df = pd.concat(dfs).reset_index().drop('index', axis = 1)
df

instring  \
0     The following are multiple choice questions (w...   
1     The following are multiple choice questions (w...   
2     The following are multiple choice questions (w...   
3     The following are multiple choice questions (w...   
4     The following are multiple choice questions (w...   
...                                                 ...   
1135  The following are multiple choice questions (w...   
1136  The following are multiple choice questions (w...   
1137  The following are multiple choice questions (w...   
1138  The following are multiple choice questions (w...   
1139  The following are multiple choice questions (w...   

                                               question  \
0     Q: A 54-year-old male presents to his primary ...   
1     Q: While on a teaching sabbatical in Uruguay, ...   
2     Q: A 45-year-old woman with hypothyroidism com...   
3     Q: A 72-year-old anthropologist with long-stan...   
4     Q: A 56-year-old woman is brought to the emerg...   
...                                                 ...   
1135  Q: A previously healthy 35-year-old primigravi...   
1136  Q: A 36-year-old man is brought to the emergen...   
1137  Q: Mutations in the ATP2A1 gene results in los...   
1138  Q: A 62-year-old man presents to the emergency...   
1139  Q: A 54-year-old man comes to the emergency de...   

                                                options  \
0     (A) Prussian Blue (B) Ziehl-Neelsen (C) Period...   
1     (A) Lidocaine (B) Midazolam (C) Halothane (D) ...   
2     (A) Mucosal cell hyperplasia (B) Enterochromaf...   
3     (A) Hyperkalemia (B) Hypernatremia (C) Thrombo...   
4     (A) Reversal of TIPS (B) Paracentesis (C) Admi...   
...                                                 ...   
1135  (A) HELLP syndrome (B) Thrombotic thrombocytop...   
1136  (A) CT scan of the chest, abdomen, and pelvis ...   
1137  (A) Muscle relaxation time: decreased, cytosol...   
1138  (A) Percutaneous coronary intervention (B) Int...   
1139  (A) Chronic inflammatory bowel disease (B) Ibu...   

                                                outputs prediction gold_answer  
0     The patient has a history of hyperlipidemia an...          D           D  
1     The patient died due to massive hepatic necros...          C           C  
2     The patient has hypothyroidism, progressive fa...          C           C  
3     The patient has a history of hypertension and ...        NaN           A  
4     The patient has increasing confusion, generali...          D           D  
...                                                 ...        ...         ...  
1135  The patient has a fever, persistent headache, ...          B           B  
1136  The patient has a gunshot wound to the chest a...          A           C  
1137  The mutation in the ATP2A1 gene results in the...          A           B  
1138  The patient has crushing chest pain that radia...          A           A  
1139  The patient has a history of psoriasis, hypert...          C           C  

[1140 rows x 6 columns]

In [17]:
len(df[df['prediction'] == df['gold_answer']])/len(df)

0.5043859649122807

In [1]:
import os
import pandas as pd

In [2]:
elems = os.listdir("/dccstor/ojasgr/scripts/approaches/forward_backward/backward_generation/llama2-70b-4Q")
# elems = elems[]

['generation_outputs-llama2-70b-4Q_800-900.csv',
 'generation_outputs-llama2-70b-4Q_500-1155.csv',
 'generation_outputs-llama2-70b-4Q_1100-1300.csv',
 'generation_outputs-llama2-70b-4Q_0-500.csv',
 'generation_outputs-llama2-70b-4Q_600-700.csv',
 'generation_outputs-llama2-70b-4Q_300-400.csv',
 'generation_outputs-llama2-70b-4Q_900-1155.csv',
 'generation_outputs-llama2-70b-4Q_1100-1155.csv',
 'generation_outputs-llama2-70b-4Q_1000-1100.csv',
 'generation_outputs-llama2-70b-4Q_700-1155.csv',
 'generation_outputs-llama2-70b-4Q_0-100.csv',
 'generation_outputs-llama2-70b-4Q_200-300.csv',
 'generation_outputs-llama2-70b-4Q_100-200.csv',
 'generation_outputs-llama2-70b-4Q_400-500.csv']

In [3]:
elems = os.listdir("/dccstor/ojasgr/scripts/approaches/forward_backward/backward_generation/llama2-70b-4Q")
elems

['generation_outputs-llama2-70b-4Q_800-900.csv',
 'generation_outputs-llama2-70b-4Q_500-1155.csv',
 'generation_outputs-llama2-70b-4Q_1100-1300.csv',
 'generation_outputs-llama2-70b-4Q_0-500.csv',
 'generation_outputs-llama2-70b-4Q_600-700.csv',
 'generation_outputs-llama2-70b-4Q_300-400.csv',
 'generation_outputs-llama2-70b-4Q_900-1155.csv',
 'generation_outputs-llama2-70b-4Q_1100-1155.csv',
 'generation_outputs-llama2-70b-4Q_1000-1100.csv',
 'generation_outputs-llama2-70b-4Q_700-1155.csv',
 'generation_outputs-llama2-70b-4Q_0-100.csv',
 'generation_outputs-llama2-70b-4Q_200-300.csv',
 'generation_outputs-llama2-70b-4Q_100-200.csv',
 'generation_outputs-llama2-70b-4Q_400-500.csv']

In [11]:
df = pd.read_csv("/dccstor/ojasgr/scripts/approaches/forward_backward/backward_generation/llama2-70b-4Q/generation_outputs-llama2-70b-4Q_1100-1155.csv")
df

instring  \
0   The following are multiple choice questions (w...   
1   The following are multiple choice questions (w...   
2   The following are multiple choice questions (w...   
3   The following are multiple choice questions (w...   
4   The following are multiple choice questions (w...   
5   The following are multiple choice questions (w...   
6   The following are multiple choice questions (w...   
7   The following are multiple choice questions (w...   
8   The following are multiple choice questions (w...   
9   The following are multiple choice questions (w...   
10  The following are multiple choice questions (w...   
11  The following are multiple choice questions (w...   
12  The following are multiple choice questions (w...   
13  The following are multiple choice questions (w...   
14  The following are multiple choice questions (w...   
15  The following are multiple choice questions (w...   
16  The following are multiple choice questions (w...   
17  The following are multiple choice questions (w...   
18  The following are multiple choice questions (w...   
19  The following are multiple choice questions (w...   
20  The following are multiple choice questions (w...   
21  The following are multiple choice questions (w...   
22  The following are multiple choice questions (w...   
23  The following are multiple choice questions (w...   
24  The following are multiple choice questions (w...   
25  The following are multiple choice questions (w...   
26  The following are multiple choice questions (w...   
27  The following are multiple choice questions (w...   
28  The following are multiple choice questions (w...   
29  The following are multiple choice questions (w...   
30  The following are multiple choice questions (w...   
31  The following are multiple choice questions (w...   
32  The following are multiple choice questions (w...   
33  The following are multiple choice questions (w...   
34  The following are multiple choice questions (w...   
35  The following are multiple choice questions (w...   
36  The following are multiple choice questions (w...   
37  The following are multiple choice questions (w...   
38  The following are multiple choice questions (w...   
39  The following are multiple choice questions (w...   

                                             question  \
0   Q: A 30-year-old woman presents to an outpatie...   
1   Q: A 1900-g (4-lb 3-oz) newborn is delivered a...   
2   Q: A 51-year-old man presents to his primary c...   
3   Q: A 68-year-old man presents to his primary c...   
4   Q: A 68-year-old postmenopausal woman presents...   
5   Q: An investigator is studying the growth patt...   
6   Q: A 7-month-old boy is brought to the physici...   
7   Q: A 4-year-old boy with acute lymphoblastic l...   
8   Q: A 27-year-old female is brought to the emer...   
9   Q: A 68-year-old man presents to the emergency...   
10  Q: A 31-year-old female patient presents with ...   
11  Q: A 44-year-old woman presents to her primary...   
12  Q: A 51-year-old Asian woman presents to her p...   
13  Q: A 49-year-old man comes to the physician be...   
14  Q: An 80-year-old African American man is brou...   
15  Q: A 4-month-old boy is brought to the pediatr...   
16  Q: A 3-year-old boy is brought to the clinic b...   
17  Q: A 26-year-old woman developed rapidly progr...   
18  Q: A 35-year-old man comes to the physician be...   
19  Q: A 33-year-old man presents to the emergency...   
20  Q: A 67-year-old man is brought to the hospita...   
21  Q: A four-day-old neonate is brought to the pe...   
22  Q: A 53-year-old male presents to his primary ...   
23  Q: A 27-year-old woman with no past medical hi...   
24  Q: A 66-year-old man comes to the physician be...   
25  Q: A 35-year-old woman presents with an unstea...   
26  Q: A 6-year-old Russian boy who recently immig...   
27  Q: A 17-year-old male presents to the emergenc...   
28  Q: A 62-year-old woman with a history of hyper... 

In [9]:
len(df[df['prediction'] == df['gold_answer']])/len(df)

0.5

In [10]:
df

instring  \
0   The following are multiple choice questions (w...   
1   The following are multiple choice questions (w...   
2   The following are multiple choice questions (w...   
3   The following are multiple choice questions (w...   
4   The following are multiple choice questions (w...   
5   The following are multiple choice questions (w...   
6   The following are multiple choice questions (w...   
7   The following are multiple choice questions (w...   
8   The following are multiple choice questions (w...   
9   The following are multiple choice questions (w...   
10  The following are multiple choice questions (w...   
11  The following are multiple choice questions (w...   
12  The following are multiple choice questions (w...   
13  The following are multiple choice questions (w...   
14  The following are multiple choice questions (w...   
15  The following are multiple choice questions (w...   
16  The following are multiple choice questions (w...   
17  The following are multiple choice questions (w...   
18  The following are multiple choice questions (w...   
19  The following are multiple choice questions (w...   
20  The following are multiple choice questions (w...   
21  The following are multiple choice questions (w...   
22  The following are multiple choice questions (w...   
23  The following are multiple choice questions (w...   
24  The following are multiple choice questions (w...   
25  The following are multiple choice questions (w...   
26  The following are multiple choice questions (w...   
27  The following are multiple choice questions (w...   
28  The following are multiple choice questions (w...   
29  The following are multiple choice questions (w...   
30  The following are multiple choice questions (w...   
31  The following are multiple choice questions (w...   
32  The following are multiple choice questions (w...   
33  The following are multiple choice questions (w...   
34  The following are multiple choice questions (w...   
35  The following are multiple choice questions (w...   
36  The following are multiple choice questions (w...   
37  The following are multiple choice questions (w...   
38  The following are multiple choice questions (w...   
39  The following are multiple choice questions (w...   

                                             question  \
0   Q: A 30-year-old woman presents to an outpatie...   
1   Q: A 1900-g (4-lb 3-oz) newborn is delivered a...   
2   Q: A 51-year-old man presents to his primary c...   
3   Q: A 68-year-old man presents to his primary c...   
4   Q: A 68-year-old postmenopausal woman presents...   
5   Q: An investigator is studying the growth patt...   
6   Q: A 7-month-old boy is brought to the physici...   
7   Q: A 4-year-old boy with acute lymphoblastic l...   
8   Q: A 27-year-old female is brought to the emer...   
9   Q: A 68-year-old man presents to the emergency...   
10  Q: A 31-year-old female patient presents with ...   
11  Q: A 44-year-old woman presents to her primary...   
12  Q: A 51-year-old Asian woman presents to her p...   
13  Q: A 49-year-old man comes to the physician be...   
14  Q: An 80-year-old African American man is brou...   
15  Q: A 4-month-old boy is brought to the pediatr...   
16  Q: A 3-year-old boy is brought to the clinic b...   
17  Q: A 26-year-old woman developed rapidly progr...   
18  Q: A 35-year-old man comes to the physician be...   
19  Q: A 33-year-old man presents to the emergency...   
20  Q: A 67-year-old man is brought to the hospita...   
21  Q: A four-day-old neonate is brought to the pe...   
22  Q: A 53-year-old male presents to his primary ...   
23  Q: A 27-year-old woman with no past medical hi...   
24  Q: A 66-year-old man comes to the physician be...   
25  Q: A 35-year-old woman presents with an unstea...   
26  Q: A 6-year-old Russian boy who recently immig...   
27  Q: A 17-year-old male presents to the emergenc...   
28  Q: A 62-year-old woman with a history of hyper... 

In [18]:
data_path = "./Updated_Test_Codex_dataset.xlsx"
data = pd.read_excel(data_path)
data

Questions  \
0     Q: A junior orthopaedic surgery resident is co...   
1     Q: A 67-year-old man with transitional cell ca...   
2     Q: Two weeks after undergoing an emergency car...   
3     Q: A 39-year-old woman is brought to the emerg...   
4     Q: A 35-year-old man comes to the physician be...   
...                                                 ...   
1150  Q: A 40-year old man presents to the clinic wi...   
1151  Q: A 36-year-old man presents to his physician...   
1152  Q: A 32-year-old woman, gravida 2, para 0, at ...   
1153  Q: A 35-year-old woman comes to the physician ...   
1154  Q: A 23-year-old man presents with swelling of...   

                                                Options  \
0     (A) Disclose the error to the patient and put ...   
1     (A) Inhibition of proteasome (B) Hyperstabiliz...   
2     (A) Renal papillary necrosis (B) Cholesterol e...   
3     (A) Coagulase-positive, gram-positive cocci fo...   
4     (A) Erythromycin ointment (B) Ketotifen eye dr...   
...                                                 ...   
1150  (A) Pancreas (B) Jejunum (C) Duodenum (D) Gall...   
1151  (A) Omeprazole interferes with bupropion metab...   
1152  (A) “I'm sorry for your loss. This must be a d...   
1153                   (A) 67% (B) 100% (C) 75% (D) 50%   
1154  (A) Metabolic inhibitor (B) Cell wall inhibito...   

                            Prompt  \
0     A: Let's think step by step.   
1     A: Let's think step by step.   
2     A: Let's think step by step.   
3     A: Let's think step by step.   
4     A: Let's think step by step.   
...                            ...   
1150  A: Let's think step by step.   
1151  A: Let's think step by step.   
1152  A: Let's think step by step.   
1153  A: Let's think step by step.   
1154  A: Let's think step by step.   

                                                  Input gold_option  
0     Q: A junior orthopaedic surgery resident is co...           B  
1     Q: A 67-year-old man with transitional cell ca...           D  
2     Q: Two weeks after undergoing an emergency car...           B  
3     Q: A 39-year-old woman is brought to the emerg...           D  
4     Q: A 35-year-old man comes to the physician be...           B  
...                                                 ...         ...  
1150  Q: A 40-year old man presents to the clinic wi...           A  
1151  Q: A 36-year-old man presents to his physician...           C  
1152  Q: A 32-year-old woman, gravida 2, para 0, at ...           A  
1153  Q: A 35-year-old woman comes to the physician ...           A  
1154  Q: A 23-year-old man presents with swelling of...           B  

[1155 rows x 5 columns]

In [19]:
reward_df = pd.read_csv("/dccstor/ojasgr/scripts/approaches/reasoning_verifier/outputs/inference/reward_model/70b-chat/medqa_test_rand500_preds_555.csv")
reward_df

instring1  \
0    Question : A 45-year-old woman comes to the ph...   
1    Question : A 75-year-old man presents to a med...   
2    Question : A 41-year-old man presents to the e...   
3    Question : A 32-year-old woman presents to the...   
4    Question : A 17-year-old high school student w...   
..                                                 ...   
495  Question : A 75-year-old woman presents to the...   
496  Question : An immunology expert is delivering ...   
497  Question : A 30-year-old primigravid woman at ...   
498  Question : A 47-year-old woman seeks evaluatio...   
499  Question : A 10-year-old girl is brought to a ...   

                                             instring2  \
0    Question : A 45-year-old woman comes to the ph...   
1    Question : A 75-year-old man presents to a med...   
2    Question : A 41-year-old man presents to the e...   
3    Question : A 32-year-old woman presents to the...   
4    Question : A 17-year-old high school student w...   
..                                                 ...   
495  Question : A 75-year-old woman presents to the...   
496  Question : An immunology expert is delivering ...   
497  Question : A 30-year-old primigravid woman at ...   
498  Question : A 47-year-old woman seeks evaluatio...   
499  Question : A 10-year-old girl is brought to a ...   

                                             instring3  \
0    Question : A 45-year-old woman comes to the ph...   
1    Question : A 75-year-old man presents to a med...   
2    Question : A 41-year-old man presents to the e...   
3    Question : A 32-year-old woman presents to the...   
4    Question : A 17-year-old high school student w...   
..                                                 ...   
495  Question : A 75-year-old woman presents to the...   
496  Question : An immunology expert is delivering ...   
497  Question : A 30-year-old primigravid woman at ...   
498  Question : A 47-year-old woman seeks evaluatio...   
499  Question : A 10-year-old girl is brought to a ...   

                                             instring4  \
0    Question : A 45-year-old woman comes to the ph...   
1    Question : A 75-year-old man presents to a med...   
2    Question : A 41-year-old man presents to the e...   
3    Question : A 32-year-old woman presents to the...   
4    Question : A 17-year-old high school student w...   
..                                                 ...   
495  Question : A 75-year-old woman presents to the...   
496  Question : An immunology expert is delivering ...   
497  Question : A 30-year-old primigravid woman at ...   
498  Question : A 47-year-old woman seeks evaluatio...   
499  Question : A 10-year-old girl is brought to a ...   

     model-ckpt-555_logits1  model-ckpt-555_logits2  model-ckpt-555_logits3  \
0                 -5.976562               -4.695312               -6.359375   
1                 -1.507812               -4.621094               -1.561523   
2                 -4.507812               -4.015625               -4.175781   
3                 -3.015625               -3.816406               -3.501953   
4                 -2.527344               -2.164062               -0.729492   
..                      ...                     ...                     ...   
495               -4.359375               -3.787109               -5.621094   
496               -3.365234                0.393311               -5.570312   
497               -4.496094               -3.691406               -3.714844   
498               -0.205933               -6.554688               -3.267578   
499               -3.150391               -4.394531               -4.171875   

     model-ckpt-555_logits4 gold_option predicted_option_555  
0                 -6.992188           D                    B  
1                 -3.041016           C                    A  
2                 -3.943359           C                    D  
3                 -2.357422           A                 

In [31]:
df = pd.read_csv("/dccstor/ojasgr/scripts/approaches/reasoning_verifier/dataset/rl_data_test.csv")
data = df.sample(500, random_state = 42).reset_index()

# Creating results on the remaining part of the dataset
data = df.merge(data, how='outer', indicator=True).query('_merge == "left_only"').drop('_merge', axis=1)

In [34]:
data.reset_index().drop('level_0',axis = 1)

questions  \
0    A junior orthopaedic surgery resident is compl...   
1    A 67-year-old man with transitional cell carci...   
2    A 35-year-old man comes to the physician becau...   
3    A 37-year-old-woman presents to her primary ca...   
4    A 72-year-old man comes to the physician becau...   
..                                                 ...   
650  A 56-year-old man comes to the emergency depar...   
651  A 40-year old man presents to the clinic with ...   
652  A 32-year-old woman, gravida 2, para 0, at 22 ...   
653  A 35-year-old woman comes to the physician for...   
654  A 23-year-old man presents with swelling of th...   

                                               options  \
0    {'A': 'Disclose the error to the patient and p...   
1    {'A': 'Inhibition of proteasome', 'B': 'Hypers...   
2    {'A': 'Erythromycin ointment', 'B': 'Ketotifen...   
3    {'A': 'A history of stroke or venous thromboem...   
4    {'A': 'Cladribine', 'B': 'Prednisone', 'C': 'I...   
..                                                 ...   
650  {'A': 'Viral myocarditis', 'B': 'Acute pericar...   
651  {'A': 'Pancreas', 'B': 'Jejunum', 'C': 'Duoden...   
652  {'A': "“I'm sorry for your loss. This must be ...   
653  {'A': '67%', 'B': '100%', 'C': '75%', 'D': '50%'}   
654  {'A': 'Metabolic inhibitor', 'B': 'Cell wall i...   

                                          gold answers  \
0    Tell the attending that he cannot fail to disc...   
1                                 Cross-linking of DNA   
2                                  Ketotifen eye drops   
3    Active or recurrent pelvic inflammatory diseas...   
4                                          Ruxolitinib   
..                                                 ...   
650               Acute inferior myocardial infarction   
651                                           Pancreas   
652  I'm sorry for your loss. This must be a diffic...   
653                                                67%   
654                                Cell wall inhibitor   

                                             instring1  \
0    Below is a question from the field of professi...   
1    Below is a question from the field of professi...   
2    Below is a question from the field of professi...   
3    Below is a question from the field of professi...   
4    Below is a question from the field of professi...   
..                                                 ...   
650  Below is a question from the field of professi...   
651  Below is a question from the field of professi...   
652  Below is a question from the field of professi...   
653  Below is a question from the field of professi...   
654  Below is a question from the field of professi...   

                                             instring2  \
0    Below is a question from the field of professi...   
1    Below is a question from the field of professi...   
2    Below is a question from the field of professi...   
3    Below is a question from the field of professi...   
4    Below is a question from the field of professi...   
..                                                 ...   
650  Below is a question from the field of professi...   
651  Below is a question from the field of professi...   
652  Below is a question from the field of professi...   
653  Below is a question from the field of professi...   
654  Below is a question from the field of professi...   

                                             instring3  \
0    Below is a question from the field of professi...   
1    Below is a question from the field of professi...   
2    Below is a question from the field of professi...   
3    Below is a question from the field of professi...   
4    Below is a question from the field of professi...   
..                                                 ...   
650  Below is a question from the field of professi...   
651  Below is a question from the field of professi...   
652  Below is a question from the fiel